In [5]:
from ngsolve import *
from ngsolve.webgui import Draw
import random
import numpy as np

#Defining parameters
epsilon = 0.02
gamma = 50
M = 1e-3
maxh = 0.01 
order = 1 
tau = 0.1            
time = 0.0
T = 1


#Defining mesh and Finite element space
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))
V = H1(mesh, order=1) 

fes = V*V #FESpace([V,V])

phi1,mu1 = fes.TrialFunction()  #n+1
v,w = fes.TestFunction()

phi_old = GridFunction(V)
#Initialising phi_0 random values between -1 and 1
random_values = np.random.uniform(-1, 1, V.ndof)
#phi_old.vec.FV().NumPy()[:] = random_values
phi_old.vec.data = random_values

#f=LinearForm(fes)
#f += tau*rhs_1*v*dx+gamma*(phi_old**3-phi_old)*w*dx+ phi_old*v*dx

A = BilinearForm(fes)
A += phi1*v*dx+tau*M*grad(mu1)*grad(v)*dx + mu1*w*dx-epsilon*grad(phi1)*grad(w)*dx
A.Assemble()

t = 0

gfu = GridFunction(fes)

gfut = GridFunction(gfu.space, multidim = 0)
gfut.AddMultiDimComponent(gfu.vec)


while t<T:
    t+=tau
    rhs = exp(-t)*x
    
    f=LinearForm(fes)
    f+= tau*rhs*v*dx+gamma*(phi_old**3-phi_old)*w*dx+ phi_old*v*dx
    f.Assemble()

    gfu.vec.data = A.mat.Inverse(freedofs = fes.FreeDofs())*f.vec

    gfut.AddMultiDimComponent(gfu.vec)
    phi_old.vec.data = gfu.components[0].vec

    
    



In [6]:
Draw(gfut.components[0], mesh, interpolate_multidim = True, animate = True)
#Draw(gfu.components[0])

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

## Fully explicit scheme

Discretize $\partial_t \phi = \frac{\phi^{n+1}-\phi^n}{\tau}$, $\mu = \mu^{n+1}$, $\phi = \phi^n$ and $\nabla \phi = \nabla \phi^{n+1}$.

$$
\begin{aligned}
(\frac{\phi^{n+1}-\phi^n}{\tau},v) + (M\nabla \mu^{n+1}, \nabla v) &= 0\ \ \ \ \forall v \in \mathbb{V} \\
(\mu^{n+1},w)-(\epsilon \nabla \phi^{n+1}, \nabla w) 
&= (\gamma ((\phi^n)^3 -\phi^n),w) \ \ \ \  \forall w \in \mathbb{V} 
\end{aligned}
$$


Kilde:https://docu.ngsolve.org/nightly/i-tutorials/unit-3.1-parabolic/parabolic.html

https://docu.ngsolve.org/latest/whetting_the_appetite/navierstokes.html